In [37]:
import pandas as pd
import numpy as np

---

- df_외감 : 2022 신용등급
- df : 우리가 가진 데이터 프레임

In [38]:
df_외감=pd.read_excel('./Dataset/외감_dp(등급).xls')
df_외감.columns = ['KIS','Stock','회사명','신용평점']

In [39]:
df=pd.read_csv('../외감기업/Dataset/외감raw3.csv',encoding='euc-kr')
df['Year']= df['회계년도'].replace('/','').astype(str).str[:4].astype(int)
df=df[df['Year']==2022]
df['자산(*)(IFRS연결)(천원)'] = df['자산(*)(IFRS연결)(천원)'] * 1000
df['log자산총계'] = np.log(df['자산(*)(IFRS연결)(천원)'])
df['잉여현금흐름(FCF)']=df['영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)']-df['투자활동으로 인한 현금흐름(*)(IFRS연결)(천원)']
df['전체현금흐름(천원)']=df['영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)']+df['투자활동으로 인한 현금흐름(*)(IFRS연결)(천원)']+df['재무활동으로 인한 현금흐름(*)(IFRS연결)(천원)']
df['총자산대비잉여현금흐름'] = df['잉여현금흐름(FCF)']/df['자산(*)(IFRS연결)(천원)']
df['총자산대비현금흐름'] = df['전체현금흐름(천원)'] / df['자산(*)(IFRS연결)(천원)']
df=df[['회사명', '거래소코드','유동자산회전률', '총자산대비잉여현금흐름','자기자본구성비율', 'log자산총계','자기자본회전률', '순운전자본회전률', '자기자본증가율', '총자본증가율', '총자산대비현금흐름', '총자본투자효율']]

c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [40]:
# df.to_csv('등급화_외감.csv')

---

- df_등급화 : merge (회사명을 기준으로)

In [41]:
df_등급화=pd.merge(df,df_외감,how='inner',on='회사명')

In [42]:
df_등급화['신용평점'].value_counts()

BBB    3732
BB     2863
B      2687
A      2053
CC     1563
CCC     692
C       554
AA      406
D       142
AAA       1
Name: 신용평점, dtype: int64

In [43]:
df_등급화.isna().sum()

회사명                0
거래소코드              0
유동자산회전률            3
총자산대비잉여현금흐름        3
자기자본구성비율           2
log자산총계            2
자기자본회전률            3
순운전자본회전률           3
자기자본증가율            2
총자본증가율             2
총자산대비현금흐름          3
총자본투자효율            2
KIS                0
Stock          14540
신용평점               0
dtype: int64

In [44]:
df_등급화=df_등급화.drop('Stock',axis=1)

In [45]:
df_등급화=df_등급화.dropna()
df_등급화 = df_등급화[(df_등급화 != 0).all(axis=1)]

In [46]:
df_등급화 = df_등급화.replace([np.inf, -np.inf], np.nan)  # inf 값을 NaN으로 대체
df_등급화 = df_등급화.dropna()  # NaN이 있는 행 삭제

In [47]:
import numpy as np

# O-Score의 계수
coefficients = {
    '유동자산회전률':0.47765086,                    # 높아야 좋다
    '총자산대비잉여현금흐름':0.53785436,            # 높아야 좋다
    '자기자본구성비율':0.49360789,                  # 높아야 좋다
    'log자산총계':0.94238478,                      # 높아야 좋다
    '자기자본회전률':-0.08646556,                   # 높아야 좋다
    '순운전자본회전률':-0.98107367,                 # 높아야 좋다
    '자기자본증가율':0.19544705,                    # 높아야 좋다
    '총자본증가율':0.44094258,                      # 높아야 좋다
    '총자산대비현금흐름':-0.37036301,               # 높아야 좋다
    '총자본투자효율':0.26458857,                    # 높아야 좋다

    'constant': 0.90495308
}

# O-Score 계산
df_등급화['dp_score'] = coefficients['constant']
df_등급화['dp_score'] += coefficients['유동자산회전률'] * df_등급화['유동자산회전률']
df_등급화['dp_score'] += coefficients['총자산대비잉여현금흐름'] * df_등급화['총자산대비잉여현금흐름']
df_등급화['dp_score'] += coefficients['자기자본구성비율'] * df_등급화['자기자본구성비율']
df_등급화['dp_score'] += coefficients['log자산총계'] * df_등급화['log자산총계']
df_등급화['dp_score'] += coefficients['자기자본회전률'] * df_등급화['자기자본회전률']
df_등급화['dp_score'] += coefficients['순운전자본회전률'] * df_등급화['순운전자본회전률']
df_등급화['dp_score'] += coefficients['자기자본증가율'] * df_등급화['자기자본증가율']
df_등급화['dp_score'] += coefficients['총자본증가율'] * df_등급화['총자본증가율']
df_등급화['dp_score'] += coefficients['총자산대비현금흐름'] * df_등급화['총자산대비현금흐름']
df_등급화['dp_score'] += coefficients['총자본투자효율'] * df_등급화['총자본투자효율']

# 로지스틱 함수를 이용해 부도 확률 계산
df_등급화['default_probability'] = 1 / (1 + np.exp(-df_등급화['dp_score']))

In [59]:
df_등급화[['dp_score','신용평점']].groupby('신용평점').agg(['mean', lambda x: x.quantile(0.25), lambda x: x.quantile(0.5), lambda x: x.quantile(0.75)])

dp_score                                    
            mean  <lambda_0>  <lambda_1>  <lambda_2>
신용평점                                                
A      76.479800   61.571516   70.448008   79.099207
AA     77.900646   66.125370   76.016799   83.888305
AAA   101.204536  101.204536  101.204536  101.204536
B      59.562975   37.393610   46.016284   58.005563
BB     57.882489   44.131724   52.074707   62.110465
BBB    70.520457   50.984459   60.492368   70.130195
C      49.292018   18.428428   31.991758   51.923569
CC     54.179345   29.591156   40.605740   55.022790
CCC    57.872817   33.630033   42.656329   53.957020
D      34.929793   11.044038   21.908953   58.561103

---

In [56]:
df_등급화.sort_values('dp_score',ascending=False)

,회사명,거래소코드,유동자산회전률,총자산대비잉여현금흐름,자기자본구성비율,log자산총계,자기자본회전률,순운전자본회전률,자기자본증가율,총자본증가율,총자산대비현금흐름,총자본투자효율,KIS,신용평점,dp_score,default_probability
8824,(주)우전지앤에프,101955,1.14,0.000260,92.44,24.378965,0.92,0.79,68061.75,274.04,3.902145e-04,2.76,V10366,BBB,13493.232778,1.000000e+00
7253,(주)에스엘정진,98574,1.65,-0.000124,21.85,24.795938,10.91,1.14,37080.30,105.25,-5.396123e-05,33.98,IT0112,B,7336.419088,1.000000e+00
11301,지엘기술(주),83372,0.91,0.000205,92.72,24.241715,0.81,0.85,1.68,0.39,-3.376076e-05,19578.39,294761,A,5249.766667,1.000000e+00
11302,지엘기술(주),83372,0.91,0.000205,92.72,24.241715,0.81,0.85,1.68,0.39,-3.376076e-05,19578.39,H90044,BBB,5249.766667,1.000000e+00
13915,평화의료정공(주),88106,2.18,-0.000033,52.19,24.120291,0.71,0.36,19340.61,134.22,-3.790851e-06,-0.38,163076,CCC,3889.171568,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,에스비아이에이에프(주),64456,234.85,0.000006,100.00,26.636652,0.20,234.71,19.93,19.93,2.374439e-07,17.93,H73167,AA,-25.313750,1.014794e-11
8510,(주)오누이,104402,2.98,-0.000441,30.47,22.851187,3.41,2.07,-82.82,-53.02,-1.445509e-03,-98.92,HO3684,BB,-29.160978,2.165450e-13
13364,(주)한국인피니티공인금거래소,94506,72.06,0.000621,3.27,21.857604,515.29,38.73,-85.91,4.74,1.500234e-05,-6.70,IN4213,B,-41.488179,9.592078e-19
418,(주)그린랩스,100214,3.11,-0.001403,16.53,24.934822,5.30,2.14,-88.24,-49.22,-1.003775e-03,-152.53,IP8045,CC,-47.817302,1.710841e-21
